In [2]:
import pandas as pd
import numpy as np
import json
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import precision_score, recall_score, average_precision_score
from Data_Processing_Quora import data_processing_quora  # تأكد أن هذا الملف يحتوي على دالة التنظيف الصحيحة

# تحميل البيانات
docs_df = pd.read_csv("quora.tsv", sep='\t').dropna(subset=["text"]).reset_index(drop=True)
queries_df = pd.read_csv("queries.tsv", sep='\t').dropna(subset=["text"])
qrels_df = pd.read_csv("qrels.tsv", sep='\t')

# بناء قاموس qrels
qrels_dict = {}
for _, row in qrels_df.iterrows():
    qid, did = row["query_id"], row["doc_id"]
    qrels_dict.setdefault(qid, set()).add(did)

# تنظيف نصوص الوثائق
print("🧹 Cleaning documents...")
docs_df["clean_text"] = docs_df["text"].apply(data_processing_quora)

# تمثيل TF-IDF للوثائق
print("🔠 Building TF-IDF representation...")
vectorizer = TfidfVectorizer(max_df=0.155, ngram_range=(1, 4), stop_words='english')
tfidf_matrix = vectorizer.fit_transform(docs_df["clean_text"])

# التقييم
map_scores, mrr_scores, precision_scores, recall_scores = [], [], [], []

print("🧪 Starting evaluation...")
for _, query_row in queries_df.iterrows():
    qid = query_row["query_id"]
    query_text = data_processing_quora(query_row["text"])
    query_vec = vectorizer.transform([query_text])
    sims = cosine_similarity(tfidf_matrix, query_vec).flatten()

    top_k = 10
    top_indices = sims.argsort()[-top_k:][::-1]
    top_doc_ids = docs_df.iloc[top_indices]["doc_id"].tolist()

    relevant_docs = qrels_dict.get(qid, set())
    y_true = [1 if doc_id in relevant_docs else 0 for doc_id in top_doc_ids]
    y_scores = sims[top_indices]

    if sum(y_true) == 0:
        continue  # تخطي الاستعلامات التي لا تحتوي على وثائق ذات صلة

    map_scores.append(average_precision_score(y_true, y_scores))
    precision_scores.append(precision_score(y_true, [1]*len(y_true)))
    recall_scores.append(recall_score(y_true, [1]*len(y_true)))
    mrr_scores.append(1 / (np.where(np.array(y_true) == 1)[0][0] + 1))

# حساب المتوسطات
map_score = np.mean(map_scores)
mrr_score = np.mean(mrr_scores)
precision_score_10 = np.mean(precision_scores)
recall_score_10 = np.mean(recall_scores)

# طباعة النتائج النهائية
print(f"\n📊 MAP: {map_score:.4f}")
print(f"📈 MRR: {mrr_score:.4f}")
print(f"🎯 P@10: {precision_score_10:.4f}")
print(f"🔍 R@10: {recall_score_10:.4f}")

# حفظ النتائج في ملف JSON
results = {
    "MAP": round(map_score, 4),
    "MRR": round(mrr_score, 4),
    "P@10": round(precision_score_10, 4),
    "R@10": round(recall_score_10, 4)
}

# عرض النتائج في النوتبوك فقط (دون حفظها في ملف)
results = {
    "MAP": round(map_score, 4),
    "MRR": round(mrr_score, 4),
    "P@10": round(precision_score_10, 4),
    "R@10": round(recall_score_10, 4)
}

# طباعة منظمة في النوتبوك

with open("evaluation_results_Quora1.json", "w", encoding="utf-8") as f:
    json.dump(results, f, indent=4)

print("\n✅ تم حفظ النتائج في evaluation_results.json")



🧹 Cleaning documents...
🔠 Building TF-IDF representation...
🧪 Starting evaluation...

📊 MAP: 0.7316
📈 MRR: 0.7741
🎯 P@10: 0.1214
🔍 R@10: 1.0000

✅ تم حفظ النتائج في evaluation_results.json
